In [1]:
import cv2
import warnings
from PIL import Image, ImageTk

from mtcnn.mtcnn import MTCNN
import tkinter as tk
import time
import numpy as np
warnings.filterwarnings("ignore")


In [2]:
from mark_attendance import RecogoniseFace
from saveImages import NewEntry
from TrainModel import TrainModel

In [3]:
class AttendanceManagement:
    def __init__(self,data_train,data_val):
        self.data_train,self.data_val =data_train,data_val
        self.rec_face = RecogoniseFace()
        self.vs = cv2.VideoCapture(0) 
        self.current_image = None  
        self.face_detector = MTCNN()
        
        
        self.root = tk.Tk()
        #self.root.geometry("400x250")  
        self.root.title("Attendance Management System")
        
        self.root.protocol('WM_DELETE_WINDOW', self.destructor)
        
        self.panel = tk.Label(self.root)  
        self.panel.pack(padx=10, pady=10)
        
        btn1 = tk.Button(self.root, text="MarkAttendance", command=self.attendance)
        btn1.pack(fill="both", expand=True, padx=10, pady=1)
        
        btn2 = tk.Button(self.root, text="NewEntry", command=self.entry)
        btn2.pack(fill="both", expand=True, padx=10, pady=1)
        
        btn3 = tk.Button(self.root, text="TrainData", command=self.train)
        btn3.pack(fill="both", expand=True, padx=10, pady=1)
        
        self.video_loop()
    
    def video_loop(self,show_name=False):
        image = self.vs.read()[1]
        self.raw_image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        if show_name:
            
            self.rec_face.recogonise(image)
            faces = self.rec_face.face_cordinates

            for i in range(len(faces)):
                x1,x2,y1,y2 = faces[i]
                cv2.rectangle(image,(x1, y1), (x2, y2), (0, 255, 0), 2)
                font = cv2.FONT_HERSHEY_SIMPLEX 
                c_name = self.rec_face.name[i]

                org = (50, 50) 
                fontScale = 1
                color = (255, 0, 0) 
                thickness = 2
                cv2.putText(image,c_name,org, font,  
                       fontScale, color, thickness, cv2.LINE_AA)
            
        self.current_image = Image.fromarray(image)  
        imgtk = ImageTk.PhotoImage(image=self.current_image)  
        self.panel.imgtk = imgtk  
        self.panel.config(image=imgtk) 
        
        self.root.after(1, self.video_loop)   
    
    def attendance(self):
        self.rec_face.recogonise(self.raw_image,mark=True)
        print("Attendance Marked")
    
    def clear(self):
        self.student_name = self.inputtxt.get("1.0", "end-1c")
        
        obj = NewEntry(self.data_train,self.data_val,self.student_name,self.new_image)
        self.inputtxt.destroy()
        self.label.destroy()
        self.clearButton.destroy()
    
            
    def entry(self):
        self.label = tk.Label(self.root,  text = "Rotate Face Slightly")
        self.label.pack(fill="both", expand=True, padx=10, pady=1)
        print("rotate head for better results")
        self.new_image = []
        i = 0
        while(i<50):
            image = self.vs.read()[1]
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            self.new_image.append(image)
            time.sleep(0.10)
            i+=1 
        self.inputtxt = tk.Text(self.root,height = 2,width = 20)
        self.inputtxt.pack()
        self.clearButton = tk.Button(self.root,text = "Submit", command = self.clear)
        self.clearButton.pack()
        
    def train(self):
        obj = TrainModel()
        obj.train(self.data_train,self.data_val)
        
    def destructor(self):
        """ Destroy the root object and release all resources """
        print("[INFO] closing...")
        self.root.destroy()
        self.vs.release()  # release web camera
        cv2.destroyAllWindows()  # it is not mandatory in this application

In [4]:
data_train,data_val = "data/train","data/val"

print("[INFO] starting...")

pba = AttendanceManagement(data_train,data_val)
pba.root.mainloop()

[INFO] starting...
Attendance Marked
rotate head for better results
[INFO] closing...
